In [4]:
from google.colab import files
import os

print("Click 'Choose Files' below and select your zip file...")
uploaded = files.upload()

# List the uploaded files
for filename in uploaded.keys():
    size_in_bytes = len(uploaded[filename])
    size_in_mb = size_in_bytes / (1024 * 1024)
    print(f'Uploaded: {filename} ({size_in_mb:.2f} MB)')

# ============================================
# YOLOv11 TRAINING - 100 EPOCHS
# ============================================

# 1. Install required packages FIRST (before importing ultralytics)
print("Installing packages...")
!pip install ultralytics pyyaml roboflow seaborn thop -q
print("✅ Packages installed\n")

# Now import the required packages
import zipfile
import yaml
from ultralytics import YOLO
import torch
import time

# 2. Check for existing zip file
print("Checking for dataset zip file...")
zip_files = [f for f in os.listdir('/content') if f.endswith('.zip')]

if not zip_files:
    print("No zip file found in /content. Please make sure you uploaded it correctly.")
    # Use the uploaded file from the initial upload
    zip_filename = list(uploaded.keys())[0]
else:
    # Use the first zip file found
    zip_filename = zip_files[0]
    print(f"Found zip file: {zip_filename}")

# 3. Extract dataset to a proper path in Colab
dataset_folder = "/content/datasets/traffic_dataset"
if os.path.exists(dataset_folder):
    !rm -rf "{dataset_folder}"

os.makedirs(dataset_folder, exist_ok=True)

print(f"Extracting {zip_filename} to {dataset_folder}...")
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

# 4. Find the actual data.yaml and fix paths
yaml_file = None
for root, dirs, files in os.walk(dataset_folder):
    if 'data.yaml' in files:
        yaml_file = os.path.join(root, 'data.yaml')
        dataset_root = root
        break

if yaml_file is None:
    raise FileNotFoundError("data.yaml not found in the extracted dataset")

print(f"Found data.yaml at: {yaml_file}")

# Read and update the data.yaml with correct Colab paths
with open(yaml_file, 'r') as f:
    data_config = yaml.safe_load(f)

# Update paths to absolute paths in Colab
data_config['train'] = os.path.join(dataset_root, 'train/images').replace('\\', '/')
data_config['val'] = os.path.join(dataset_root, 'valid/images').replace('\\', '/')
if 'test' in data_config and data_config['test']:
    data_config['test'] = os.path.join(dataset_root, 'test/images').replace('\\', '/')

# Also fix any path separators in the names
data_config['path'] = dataset_root.replace('\\', '/')

# Save the updated data.yaml
with open(yaml_file, 'w') as f:
    yaml.dump(data_config, f, default_flow_style=False)

print("✅ Dataset prepared and data.yaml updated.")
print(f"Train path: {data_config['train']}")
print(f"Val path: {data_config['val']}")

# 5. Check GPU
print(f"\nGPU available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 6. Train the model
start_time = time.time()

# Load a model
model = YOLO('yolo11m.pt')  # Using medium model

# Train
results = model.train(
    data=yaml_file,
    epochs=100,
    imgsz=640,
    batch=16,
    patience=15,
    project='/content/runs',
    name='traffic_100epochs',
    exist_ok=True,
    device=0 if torch.cuda.is_available() else 'cpu',
    verbose=True,
)

# Print training time
end_time = time.time()
print(f"Training completed in {(end_time - start_time)/3600:.2f} hours.")

# 7. Evaluate the best model
best_model_path = '/content/runs/traffic_100epochs/weights/best.pt'
if os.path.exists(best_model_path):
    best_model = YOLO(best_model_path)
    metrics = best_model.val()

    # Print metrics
    print(f"\nValidation Results:")
    print(f"Precision: {metrics.box.mp:.4f}")
    print(f"Recall: {metrics.box.mr:.4f}")
    print(f"mAP@0.5: {metrics.box.map50:.4f}")
    print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")

    # Per-class metrics
    print("\nPer-class metrics:")
    for i, class_name in enumerate(metrics.names.values()):
        if i < len(metrics.box.p):
            print(f"  {class_name}:")
            print(f"    Precision: {metrics.box.p[i]:.4f}")
            print(f"    Recall: {metrics.box.r[i]:.4f}")
            print(f"    mAP50: {metrics.box.ap50[i]:.4f}")
else:
    print(f"Best model not found at {best_model_path}")

Click 'Choose Files' below and select your zip file...


Saving Wireless Int Modified.v2i.yolov11.zip to Wireless Int Modified.v2i.yolov11 (1).zip
Uploaded: Wireless Int Modified.v2i.yolov11 (1).zip (7.40 MB)
Installing packages...
✅ Packages installed

Checking for dataset zip file...
Found zip file: Wireless Int Modified.v2i.yolov11.zip
Extracting Wireless Int Modified.v2i.yolov11.zip to /content/datasets/traffic_dataset...
Found data.yaml at: /content/datasets/traffic_dataset/data.yaml
✅ Dataset prepared and data.yaml updated.
Train path: /content/datasets/traffic_dataset/train/images
Val path: /content/datasets/traffic_dataset/valid/images

GPU available: True
GPU: Tesla T4
Ultralytics 8.4.14 🚀 Python-3.12.12 torch-2.9.0+cu128 CUDA:0 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/con

In [5]:
import os

weights_dir = '/content/runs/traffic_100epochs/weights/'

if os.path.exists(weights_dir):
    print(f"Contents of {weights_dir}:")
    for item in os.listdir(weights_dir):
        print(f"- {item}")
else:
    print(f"Directory not found: {weights_dir}")

Contents of /content/runs/traffic_100epochs/weights/:
- best.pt
- last.pt


In [ ]:
from google.colab import drive
drive.mount('/content/drive')